In [1]:
import pandas as pd

from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
contract_url= 'https://www.spotrac.com/mlb/contracts/'
war_url= 'https://www.fangraphs.com/warleaders.aspx?season=2019&team=all'
war_csv = "./FanGraphs Leaderboard.csv"

In [3]:
contracts =pd.read_html(contract_url)

In [4]:
#war =pd.read_html(war_url)
war =pd.read_csv(war_csv)

In [5]:
contracts[0].head()

,Player,Pos,Team,Age,Yrs,Dollars,Average,Free Agent
0,Mike Trout (2019-2030),CF,LAA,28,12,"$426,500,000","$35,541,667",2031
1,Bryce Harper (2019-2031),RF,PHI,26,13,"$330,000,000","$25,384,615",2032
2,Giancarlo Stanton (2015-2027),DH,NYY Signed w/MIA,29,13,"$325,000,000","$25,000,000",2029
3,Manny Machado (2019-2028),3B,SD,27,10,"$300,000,000","$30,000,000",2029
4,Nolan Arenado (2019-2026),3B,COL,28,8,"$260,000,000","$32,500,000",2027


In [6]:
war.head()

,Name,playerid,Team,Pos,PA,IP,Primary WAR,Total WAR
0,Mike Trout,10155,Angels,CF,600.0,NaN,8.649976,8.6
1,Alex Bregman,17678,Astros,3B/SS,690.0,NaN,8.486207,8.5
2,Christian Yelich,11477,Brewers,RF,580.0,NaN,7.778790,7.8
3,Cody Bellinger,15998,Dodgers,RF,660.0,NaN,7.769759,7.8
4,Jacob deGrom,10954,Mets,P,NaN,204.0,7.003128,7.6


In [7]:
contract_df = contracts[0]
war_df = war

In [8]:
#contract_df.to_csv("./Contracts.csv")

In [9]:
#Dataframe had a ridiculous name so we saved it to csv to rename and then redownloaded it
#ended up not needing to do this because fansided already had a full data csv

#war_df.to_csv("./WAR.csv")

In [10]:
#new_war_df = pd.read_csv("./WAR.csv")

In [11]:
contract_df.dropna(inplace=True)

In [12]:
#This was to fix names for old data

#list(new_war_df)
#new_war_df.drop(['Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12'], axis=1, inplace=True)

In [13]:
#new_war_df.head()

In [14]:
list(contract_df)

['Player', 'Pos', 'Team', 'Age', 'Yrs', 'Dollars', 'Average', 'Free Agent']

In [15]:
new = contract_df["Player"].str.split(" ", n = 0, expand = True) 

In [16]:
full_name =new[0].astype(str) +' '+ new[1].astype(str)

In [17]:
contract_df["Name"] = full_name
contract_df["contract_years"] = new[2]
contract_df.drop(columns=['Player'], inplace=True)

In [18]:
contract_df.head()

,Pos,Team,Age,Yrs,Dollars,Average,Free Agent,Name,contract_years
0,CF,LAA,28,12,"$426,500,000","$35,541,667",2031,Mike Trout,(2019-2030)
1,RF,PHI,26,13,"$330,000,000","$25,384,615",2032,Bryce Harper,(2019-2031)
2,DH,NYY Signed w/MIA,29,13,"$325,000,000","$25,000,000",2029,Giancarlo Stanton,(2015-2027)
3,3B,SD,27,10,"$300,000,000","$30,000,000",2029,Manny Machado,(2019-2028)
4,3B,COL,28,8,"$260,000,000","$32,500,000",2027,Nolan Arenado,(2019-2026)


In [19]:
new_df = contract_df.merge(war_df, left_on ="Name", right_on="Name")

In [20]:
new_df.drop(columns= ["Pos_x","Team_x"], inplace=True)

In [21]:
new_df.head()

,Age,Yrs,Dollars,Average,Free Agent,Name,contract_years,playerid,Team_y,Pos_y,PA,IP,Primary WAR,Total WAR
0,28,12,"$426,500,000","$35,541,667",2031,Mike Trout,(2019-2030),10155,Angels,CF,600.0,NaN,8.649976,8.6
1,26,13,"$330,000,000","$25,384,615",2032,Bryce Harper,(2019-2031),11579,Phillies,RF,682.0,NaN,4.562504,4.6
2,29,13,"$325,000,000","$25,000,000",2029,Giancarlo Stanton,(2015-2027),4949,Yankees,LF/DH,72.0,NaN,0.360692,0.4
3,27,10,"$300,000,000","$30,000,000",2029,Manny Machado,(2019-2028),11493,Padres,3B,661.0,NaN,3.076305,3.1
4,28,8,"$260,000,000","$32,500,000",2027,Nolan Arenado,(2019-2026),9777,Rockies,3B,662.0,NaN,5.929520,5.9


In [22]:
new_df.rename(columns={"Team_y":"Team","Pos_y":"Pos"}, inplace=True)

In [23]:
new_df = new_df[['playerid',
                'Name',
                'Team',
                 'Pos',
                'Age',
 'Dollars',
 'Average',
 'Free Agent',
 'contract_years',
 'Yrs',
 'PA',
 'IP',
 'Primary WAR',
 'Total WAR']]

In [24]:
new_df["Total WAR"].quantile([.25,.5,.75])

0.25   -0.1
0.50    0.4
0.75    1.4
Name: Total WAR, dtype: float64

In [25]:
new_df['Int Average']= new_df['Average']

In [26]:
new_df['Int Average'] = new_df['Int Average'].str.replace(',', '')
new_df['Int Average'] = new_df['Int Average'].str.replace('$', '')
new_df['Int Average'] = new_df['Int Average'].astype(int)

In [27]:
new_df.head()

,playerid,Name,Team,Pos,Age,Dollars,Average,Free Agent,contract_years,Yrs,PA,IP,Primary WAR,Total WAR,Int Average
0,10155,Mike Trout,Angels,CF,28,"$426,500,000","$35,541,667",2031,(2019-2030),12,600.0,NaN,8.649976,8.6,35541667
1,11579,Bryce Harper,Phillies,RF,26,"$330,000,000","$25,384,615",2032,(2019-2031),13,682.0,NaN,4.562504,4.6,25384615
2,4949,Giancarlo Stanton,Yankees,LF/DH,29,"$325,000,000","$25,000,000",2029,(2015-2027),13,72.0,NaN,0.360692,0.4,25000000
3,11493,Manny Machado,Padres,3B,27,"$300,000,000","$30,000,000",2029,(2019-2028),10,661.0,NaN,3.076305,3.1,30000000
4,9777,Nolan Arenado,Rockies,3B,28,"$260,000,000","$32,500,000",2027,(2019-2026),8,662.0,NaN,5.929520,5.9,32500000


In [28]:
new_df['Dollars per WAR'] = (new_df['Int Average']/100000)/new_df['Total WAR']

In [29]:
new_df.to_csv("./new_contracts.csv")

In [30]:
table_list = []

In [31]:
test_table = pd.read_html(war_url)
test_table = test_table[4]

In [66]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [67]:
browser.visit(war_url)

In [69]:
#There is a popup asking for a membership that has no rhyme or reason to it, but will break the scraping.
#We have only had it pop up a few times.As long as it doesn't pop up it works great
#I think we fixed this so it clicks through the pop up - had to make the sleep timer fairly long to let the ad load


data = []
for x in range(1,55):
    time.sleep(5)
    html = browser.html
    soup = bs(html, "html.parser") 
#    data.append(pd.read_html(str(soup))[12])
    try:
        time.sleep(2)
        browser.click_link_by_partial_text('No Thanks')
    except:
        data.append(pd.read_html(str(soup))[12])
        try:
#    browser.click_link_by_text(x+1)
            browser.find_by_name('WARBoard1$dg1$ctl00$ctl02$ctl00$ctl18').first.click()
#         browser.click_link_by_text(str(x))
        except:
            browser.find_by_name('WARBoard1$dg1$ctl00$ctl02$ctl00$ctl20').first.click()
#         browser.click_link_by_text(str(x))



WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.17763 x86_64)


In [ ]:
len(data)

12345678910... Page size: select  1410 items in 47 pages                    \
                                                         #              Name   
0                                                  1              Mike Trout   
1                                                  2            Alex Bregman   
2                                                  3        Christian Yelich   
3                                                  4          Cody Bellinger   
4                                                  5            Jacob deGrom   

                                                     
      Team    Pos   PA     IP Primary WAR Total WAR  
0   Angels     CF  600    NaN         8.6       8.6  
1   Astros  3B/SS  690    NaN         8.5       8.5  
2  Brewers     RF  580    NaN         7.8       7.8  
3  Dodgers     RF  660    NaN         7.8       7.8  
4     Mets      P  NaN  204.0         7.0       7.6

In [204]:
crap_shoot = data[0]
for x in range(1,len(data)):
    try: 
        crap_shoot = crap_shoot.append(data[x])
    except:
        print("error")

In [206]:
crap_shoot.to_csv("./crap_shoot.csv")

In [145]:
data[1]

12345678910... Page size: select  1410 items in 47 pages  \
                                                          #   
0                                                  31         
1                                                  32         
2                                                  33         
3                                                  34         
4                                                  35         
5                                                  36         
6                                                  37         
7                                                  38         
8                                                  39         
9                                                  40         
10                                                 41         
11                                                 42         
12                                                 43         
13                                                 44         
14                                                 45         
15                                                 46         
16                                                 47         
17                                                 48         
18                                                 49         
19                                                 50         
20                                                 51         
21                                                 52         
22                                                 53         
23                                                 54         
24                                                 55         
25                                                 56         
26                                                 57         
27                                                 58         
28                                                 59         
29                                                 60         
30  12345678910... Page size: select  1410 items i...         

                                                       \
                                                 Name   
0                                       Lucas Giolito   
1                                       Jack Flaherty   
2                                      Josh Donaldson   
3                                      Walker Buehler   
4                                        Peter Alonso   
5                                           Juan Soto   
6                                           Max Muncy   
7                                         Kris Bryant   
8                                        Ozzie Albies   
9                                         Aaron Judge   
10                                       Bryce Harper   
11                                        Jeff McNeil   
12                                     Patrick Corbin   
13                                     Mike Clevinger   
14                                     Eugenio Suarez   
15                                         Sonny Gray   
16                                         Max Kepler   
17                                   Noah Syndergaard   
18                                       Jose Berrios   
19                                        Javier Baez   
20                                   Francisco Lindor   
21                                     Carlos Santana   
22                                      Jake Odorizzi   
23                                        Nelson Cruz   
24                                     Kyle Hendricks   
25                                   Michael Brantley   
26                                        Paul DeJong   
27                                      Jorge Polanco   
28                                     Austin Meadows   
29                                         Mike Minor   
30  12345678910... Page size: select  1410 items i...   

                                         

In [31]:
cleaned_csv = './crap_shoot_clean_all.csv'
cleaned_war =pd.read_csv(war_csv)
cleaned_war.fillna(0,inplace=True)

In [32]:
cleaned_war = cleaned_war[['playerid',
                'Name',
                'Team',
                 'Pos',
                 'PA',
                 'IP',
                 'Primary WAR',
                 'Total WAR']]

cleaned_war.head()

,playerid,Name,Team,Pos,PA,IP,Primary WAR,Total WAR
0,10155,Mike Trout,Angels,CF,600.0,0.0,8.649976,8.6
1,17678,Alex Bregman,Astros,3B/SS,690.0,0.0,8.486207,8.5
2,11477,Christian Yelich,Brewers,RF,580.0,0.0,7.778790,7.8
3,15998,Cody Bellinger,Dodgers,RF,660.0,0.0,7.769759,7.8
4,10954,Jacob deGrom,Mets,P,0.0,204.0,7.003128,7.6


In [33]:
updated_contracts = new_df.drop(columns= ["PA","IP","Primary WAR", "Total WAR", "Int Average","Dollars per WAR"])

In [36]:
updated_contracts.head()

,playerid,Name,Team,Pos,Age,Dollars,Average,Free Agent,contract_years,Yrs
0,10155,Mike Trout,Angels,CF,28,"$426,500,000","$35,541,667",2031,(2019-2030),12
1,11579,Bryce Harper,Phillies,RF,26,"$330,000,000","$25,384,615",2032,(2019-2031),13
2,4949,Giancarlo Stanton,Yankees,LF/DH,29,"$325,000,000","$25,000,000",2029,(2015-2027),13
3,11493,Manny Machado,Padres,3B,27,"$300,000,000","$30,000,000",2029,(2019-2028),10
4,9777,Nolan Arenado,Rockies,3B,28,"$260,000,000","$32,500,000",2027,(2019-2026),8


In [37]:
final_df = updated_contracts.merge(cleaned_war, left_on ="playerid", right_on="playerid")
final_df.drop(columns= ["Pos_x","Team_x","Name_x"], inplace=True)
final_df.rename(columns={"Team_y":"Team","Pos_y":"Pos","Name_y":"Name"}, inplace=True)

In [38]:
final_df = final_df[['playerid',
                'Name',
                'Team',
                 'Pos',
                'Age',
 'Dollars',
 'Average',
 'Free Agent',
 'contract_years',
 'Yrs',
 'PA',
 'IP',
 'Primary WAR',
 'Total WAR']]

In [39]:
final_df['Int Average']= final_df['Average']

final_df['Int Average'] = final_df['Int Average'].str.replace(',', '')
final_df['Int Average'] = final_df['Int Average'].str.replace('$', '')
final_df['Int Average'] = final_df['Int Average'].astype(int)

In [40]:
final_df['Dollars per WAR(100K)'] = (final_df['Int Average']/100000)/new_df['Total WAR']

In [41]:
final_df.head()

,playerid,Name,Team,Pos,Age,Dollars,Average,Free Agent,contract_years,Yrs,PA,IP,Primary WAR,Total WAR,Int Average,Dollars per WAR(100K)
0,10155,Mike Trout,Angels,CF,28,"$426,500,000","$35,541,667",2031,(2019-2030),12,600.0,0.0,8.649976,8.6,35541667,41.327520
1,11579,Bryce Harper,Phillies,RF,26,"$330,000,000","$25,384,615",2032,(2019-2031),13,682.0,0.0,4.562504,4.6,25384615,55.183946
2,4949,Giancarlo Stanton,Yankees,LF/DH,29,"$325,000,000","$25,000,000",2029,(2015-2027),13,72.0,0.0,0.360692,0.4,25000000,625.000000
3,11493,Manny Machado,Padres,3B,27,"$300,000,000","$30,000,000",2029,(2019-2028),10,661.0,0.0,3.076305,3.1,30000000,96.774194
4,9777,Nolan Arenado,Rockies,3B,28,"$260,000,000","$32,500,000",2027,(2019-2026),8,662.0,0.0,5.929520,5.9,32500000,55.084746


In [42]:
final_df.to_csv("./final_df.csv")